In [1]:
import pandas as pd
import json
import numpy as np
import cv2

In [2]:
all_documents=pd.read_csv('query_result_2022-02-03T11_48_29.164127Z.csv')

In [3]:
all_documents

,case_id,image_key,bucket,status,id,image_id,class,visibility,type,meta,parent_id,deleted_at,created_at,updated_at,polygon
0,1cd90041-b111-4063-b576-6d549c15d210,images_zippy/document_images/a5ea4c25-dc13-418...,rz-dre-weights,MERGED,5398148,817440,NaN,around_75,profile,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",NaN,NaN,2022-01-18T16:51:30.713466Z,2022-01-18T16:51:30.713466Z,"[{""x"": 294.9794238683128, ""y"": 208.06584362139..."
1,1cd90041-b111-4063-b576-6d549c15d210,images_zippy/document_images/a5ea4c25-dc13-418...,rz-dre-weights,MERGED,5398147,817439,NaN,around_50,profile,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",NaN,NaN,2022-01-18T16:49:21.053716Z,2022-01-18T16:49:21.053716Z,"[{""x"": 287.0781893004115, ""y"": 165.92592592592..."
2,1cd90041-b111-4063-b576-6d549c15d210,images_zippy/document_images/a5c86375-7bdd-4de...,rz-dre-weights,MERGED,5398146,817438,NaN,fully_visible,profile,"{""state"": ""Haryana"", ""documentSide"": ""Front"", ...",NaN,NaN,2022-01-18T16:45:53.807403Z,2022-01-18T16:45:53.807403Z,"[{""x"": 229.908592321755, ""y"": 203.583180987202..."
3,1cd90041-b111-4063-b576-6d549c15d210,images_zippy/document_images/a5b3d82f-0ba3-4df...,rz-dre-weights,MERGED,5398145,817437,NaN,around_75,profile,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",NaN,NaN,2022-01-18T16:43:07.688191Z,2022-01-18T16:43:07.688191Z,"[{""x"": 551.0786106032906, ""y"": 328.19012797074..."
4,1cd90041-b111-4063-b576-6d549c15d210,images_zippy/document_images/a5b3d82f-0ba3-4df...,rz-dre-weights,MERGED,5398141,817436,NaN,fully_visible,profile,"{""state"": ""Delhi"", ""documentSide"": ""back"", ""do...",NaN,NaN,2022-01-18T16:40:51.909386Z,2022-01-18T16:40:51.909386Z,"[{""x"": 542.3034734917733, ""y"": 257.98903107861..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,861ba1d1-2060-4ba0-9c84-12ed7b20269b,images_zippy/document_images/3c8878b5-22d4-434...,rz-dre-weights,MERGED,5310605,822223,NaN,around_75,profile,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",NaN,NaN,2022-01-11T11:42:12.740376Z,2022-01-11T11:42:12.740376Z,"[{""x"": 406.7163288040666, ""y"": 281.14789748033..."
4911,861ba1d1-2060-4ba0-9c84-12ed7b20269b,images_zippy/document_images/3c768cea-79aa-44d...,rz-dre-weights,MERGED,5310593,822222,NaN,around_75,profile,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",NaN,NaN,2022-01-11T11:41:37.290455Z,2022-01-11T11:41:37.290455Z,"[{""x"": 454.39844800936464, ""y"": 303.3995531094..."
4912,861ba1d1-2060-4ba0-9c84-12ed7b20269b,images_zippy/document_images/3c65bae5-fa74-407...,rz-dre-weights,MERGED,5310582,822221,NaN,fully_visible,profile,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",NaN,NaN,2022-01-11T11:41:06.710899Z,2022-01-11T11:41:06.710899Z,"[{""x"": 416.25275264512624, ""y"": 193.7306789372..."
4913,861ba1d1-2060-4ba0-9c84-12ed7b20269b,images_zippy/document_images/3c65bae5-fa74-407...,rz-dre-weights,MERGED,5310574,822220,NaN,fully_visible,profile,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",NaN,NaN,2022-01-11T11:40:41.524001Z,2022-01-11T11:40:41.524001Z,"[{""x"": 293.86864668486135, ""y"": 157.1743875465..."


In [4]:
case_id = all_documents["case_id"]
image_id = all_documents["image_id"]
visibility = all_documents["visibility"]
meta = all_documents["meta"]
polygon = all_documents["polygon"]
id1 = all_documents["id"]


In [9]:
df = pd.DataFrame(columns=['file_name', 'class', 'Width', 'Height', 'xmin', 'ymin','xmax','ymax' ,'visibility','meta', 'class_id' , 'case_id', 'image_id', 'visibility', 'polygon', 'id'])

In [10]:
def genrate_pts(poly_line):
    res = json.loads(poly_line)
    #print(res)
    coordinates = []
    for j in range(len(res)):
        a = res[j]['x']
        b = res[j]['y']
        c = [int(a) , int(b)]
        coordinates.append(c)
    pts = np.array(coordinates)
    return pts



In [11]:
def bounding_box(points):
    x_coordinates, y_coordinates = zip(*points)

    return [(min(x_coordinates), min(y_coordinates)), (max(x_coordinates), max(y_coordinates))]

In [12]:
for i in range(len(image_id)):
    image_name = all_documents.iloc[i]["case_id"]+"_"+str(all_documents.iloc[i]["image_id"])+".jpg"
    poly_line = all_documents["polygon"][i]
    image_path = 'card_images/'+image_name
    image = cv2.imread(image_path)
    height, width, channels = image.shape
    #print(height, width, channels)
    pts = genrate_pts(poly_line)
    
    bbox = bounding_box(pts)
    #print(bbox[0][0], bbox[0][1], bbox[1][0], bbox[1][1])
    df = df.append({'file_name' : image_name, 'class' : 'RC','Width' :  width, 'Height' : height, 'xmin' : bbox[0][0], 'ymin' : bbox[0][1],'xmax' : bbox[1][0] ,'ymax' : bbox[1][1],'visibility' : visibility[i] ,'meta' : meta[i], 'class_id' : 1, 'case_id' : case_id[i]  , 'image_id' : image_id[i] , 'visibility' : visibility[i], 'polygon' : polygon[i] , 'id' : id1[i] },ignore_index=True)

df

,file_name,class,Width,Height,xmin,ymin,xmax,ymax,visibility,meta,class_id,case_id,image_id,visibility,polygon,id
0,1cd90041-b111-4063-b576-6d549c15d210_817440.jpg,RC,1920,1080,215,163,1566,958,around_75,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",1,1cd90041-b111-4063-b576-6d549c15d210,817440,around_75,"[{""x"": 294.9794238683128, ""y"": 208.06584362139...",5398148
1,1cd90041-b111-4063-b576-6d549c15d210_817439.jpg,RC,1920,1080,205,134,1659,985,around_50,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",1,1cd90041-b111-4063-b576-6d549c15d210,817439,around_50,"[{""x"": 287.0781893004115, ""y"": 165.92592592592...",5398147
2,1cd90041-b111-4063-b576-6d549c15d210_817438.jpg,RC,1920,1080,214,164,1491,979,fully_visible,"{""state"": ""Haryana"", ""documentSide"": ""Front"", ...",1,1cd90041-b111-4063-b576-6d549c15d210,817438,fully_visible,"[{""x"": 229.908592321755, ""y"": 203.583180987202...",5398146
3,1cd90041-b111-4063-b576-6d549c15d210_817437.jpg,RC,1920,1080,540,301,1400,838,around_75,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",1,1cd90041-b111-4063-b576-6d549c15d210,817437,around_75,"[{""x"": 551.0786106032906, ""y"": 328.19012797074...",5398145
4,1cd90041-b111-4063-b576-6d549c15d210_817436.jpg,RC,1920,1080,489,238,1333,800,fully_visible,"{""state"": ""Delhi"", ""documentSide"": ""back"", ""do...",1,1cd90041-b111-4063-b576-6d549c15d210,817436,fully_visible,"[{""x"": 542.3034734917733, ""y"": 257.98903107861...",5398141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,861ba1d1-2060-4ba0-9c84-12ed7b20269b_822223.jpg,RC,1920,1080,406,222,1417,862,around_75,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",1,861ba1d1-2060-4ba0-9c84-12ed7b20269b,822223,around_75,"[{""x"": 406.7163288040666, ""y"": 281.14789748033...",5310605
4911,861ba1d1-2060-4ba0-9c84-12ed7b20269b_822222.jpg,RC,1920,1080,454,273,1249,783,around_75,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",1,861ba1d1-2060-4ba0-9c84-12ed7b20269b,822222,around_75,"[{""x"": 454.39844800936464, ""y"": 303.3995531094...",5310593
4912,861ba1d1-2060-4ba0-9c84-12ed7b20269b_822221.jpg,RC,1920,1080,381,146,1597,921,fully_visible,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",1,861ba1d1-2060-4ba0-9c84-12ed7b20269b,822221,fully_visible,"[{""x"": 416.25275264512624, ""y"": 193.7306789372...",5310582
4913,861ba1d1-2060-4ba0-9c84-12ed7b20269b_822220.jpg,RC,1920,1080,230,112,1533,916,fully_visible,"{""state"": ""Delhi"", ""documentSide"": ""Front"", ""d...",1,861ba1d1-2060-4ba0-9c84-12ed7b20269b,822220,fully_visible,"[{""x"": 293.86864668486135, ""y"": 157.1743875465...",5310574


In [13]:
df.to_csv('rc_card_polygon_data.csv', index = False)